## Feature extraction Cpfl1 - porespy

The workflow to demonstrate the feature extraction with the function `regionprops_3D` from `porespy` was already demonstrated [here]().

This notebook aims to extract the same features from all images the Cpfl1 dataset with a for loop.

In [1]:
import apoc
import numpy as np
import os
import pandas as pd
import pyclesperanto_prototype as cle

from porespy.metrics import regionprops_3D
from skimage.measure import regionprops_table

import sys
sys.path.append("../..")
from quapos_lm import rescale_image, rescale_segmentation, predict_image

In [2]:
# Load classifier
quapos_lm = apoc.ObjectSegmenter(opencl_filename = "../../01-training-and-validation/02-quapos-lm.cl")
quapos_lm.feature_importances()

{'gaussian_blur=1': 0.32557488170342097,
 'difference_of_gaussian=1': 0.4231073391932076,
 'laplace_box_of_gaussian_blur=1': 0.25131777910337144}

In [3]:
# Define file path for images
images = "../../data/02-data-for-pixel-classifier/cpfl-wt-comparison/b-s-ops-images-cropped/"

# Define a file list from the folder
file_list = os.listdir(images)
print(file_list)

['C2-cpfl-p08-1.3-20x-flo.tif', 'C2-cpfl-p08-2.3-20x-flo.tif', 'C2-cpfl-p08-3.3-20x-flo.tif', 'C2-cpfl-p14-1.3-20x-flo.tif', 'C2-cpfl-p14-2.3-20x-flo.tif', 'C2-cpfl-p14-3.3-20x-flo.tif', 'C2-cpfl-p20-1.3-20x-flo.tif', 'C2-cpfl-p20-2.3-20x-flo.tif', 'C2-cpfl-p20-3.3-20x-flo.tif', 'C2-cpfl-p245-1.3-20x-flo.tif', 'C2-cpfl-p245-2.3-20x-flo.tif', 'C2-cpfl-p245-3.3-20x-flo.tif', 'C2-cpfl-p245-4.3-20x-flo.tif', 'C2-cpfl-p30-1.4-20x-flo.tif', 'C2-cpfl-p30-2.4-20x-flo.tif', 'C2-cpfl-p30-3.4-20x-flo.tif', 'C2-cpfl-p70-1.4-20x-flo.tif', 'C2-cpfl-p70-2.4-20x-flo.tif', 'C2-cpfl-p70-3.4-20x-flo.tif', 'C2-wt-p08-1.4-20x-flo.tif', 'C2-wt-p08-2.4-20x-flo.tif', 'C2-wt-p08-3.4-20x-flo.tif', 'C2-wt-p08-4.4-20x-flo.tif', 'C2-wt-p14-1.4-20x-flo.tif', 'C2-wt-p14-2.4-20x-flo.tif', 'C2-wt-p14-3.4-20x-flo.tif', 'C2-wt-p14-4.4-20x-flo.tif', 'C2-wt-p20-1.4-20x-flo.tif', 'C2-wt-p20-1.4-20x-suse.tif', 'C2-wt-p20-2.4-20x-flo.tif', 'C2-wt-p20-2.4-20x-suse.tif', 'C2-wt-p245-1.1-20x-flo.tif', 'C2-wt-p245-2.1-20x-flo.ti

### Extract features

Now a for loop will be computed to extract all the features.

In [5]:
# Define an empty array to store all the data
features_porespy = []

# Loop over the image folder and extract all the features
for i, file_name in enumerate(file_list):
    
    # Load image
    image = cle.imread(images + file_name)
    
    # Predict the image
    prediction = predict_image(image=image, classifier=quapos_lm)
    
    # Rescale the image
    image_rescaled = rescale_image(image=image, voxel_x=0.323, voxel_y=0.323, voxel_z=0.490)
    
    # Rescaled the prediction
    prediction_rescaled = rescale_segmentation(segmentation=prediction, voxel_x=0.323, voxel_y=0.323, voxel_z=0.490)
        
    # Feature extraction with porespy regionprops_3D
    features_i = regionprops_3D(prediction_rescaled)
    
    # Define empty array to store features of current image
    extracted_features = []
    
    # Loop over the extracted features to obtain features in accessible data table
    for prop in features_i:
        
        # Obtain some features
        label = prop.label
        surface_area = prop.surface_area
        volume = prop.volume
        bbox_volume = prop.bbox_volume
        convex_volume = prop.convex_volume
        sphericity = prop.sphericity
        solidity = prop.solidity
        
        # Obtain the predicted label in its bounding box, flatten it into 2D and obtain more features
        mask = prop.mask
        mask = cle.maximum_z_projection(mask)
        mask = cle.connected_components_labeling_diamond(mask)
        
        # From 2D label major axis length retrieved
        major_axis_length = regionprops_table(
            label_image = mask,
            properties = ["axis_major_length"])
        major_axis_length = major_axis_length["axis_major_length"][0]
        
        # Minor axis length retrieved
        minor_axis_length = regionprops_table(
            label_image = mask,
            properties = ["axis_minor_length"])
        minor_axis_length = minor_axis_length["axis_minor_length"][0]
        
        # Crofton perimeter retrieved
        perimeter_crofton = regionprops_table(
            label_image = mask,
            properties = ["perimeter_crofton"])
        perimeter_crofton = perimeter_crofton["perimeter_crofton"][0]
        
        # Define names for the different variables
        measurements_dict = {"label": label,
                             "surface_area": surface_area,
                             "volume": volume,
                             "bbox_volume": bbox_volume,
                             "convex_volume": convex_volume,
                             "sphericity": sphericity,
                             "solidity": solidity,
                             "perimeter_2d": perimeter_crofton,
                             "major_axis_length_2d": major_axis_length,
                             "minor_axis_length_2d": minor_axis_length}
        
        # Add features of current image into dataframe
        extracted_features.append(measurements_dict)
    
    # Turn features of current image into a pandas dataframe
    extracted_features = pd.DataFrame(extracted_features)
    
    # Add features of current image into final dataframe
    features_porespy.append(extracted_features)

C:\Users\fsalo\mambaforge\envs\outer-segment-analysis\lib\site-packages\skimage\measure\_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1870371730  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  2  Error-roundoff 2.5e-15  _one-merge 1.7e-14
  _near-inside 8.6e-14  Visible-distance 4.9e-15  U-max-coplanar 4.9e-15
  Width-outside 9.8e-15  _wide-facet 3e-14  _maxoutside 2e-14

precision problems (corrected unless 'Q0' or an error)
      1 degenerate hyperplanes recomputed with gaussian elimination
      1 nearly singular or axis-parallel hyperplanes
      1 zero divisors during back substitute
      1 zero divisors during gaussian elimination

The input to qhull appears to be less than 3 dimensional, or a
computation ha

In [6]:
features_porespy = pd.concat(features_porespy)

In [8]:
features_porespy

,label,surface_area,volume,bbox_volume,convex_volume,sphericity,solidity,perimeter_2d,major_axis_length_2d,minor_axis_length_2d
0,1,37.206985,29.0,45,32,1.226850,0.906250,9.155272,3.023716,2.799417
1,2,68.012772,95.0,144,106,1.480390,0.896226,15.398989,6.585671,3.179608
2,3,10.392303,5.0,6,0,1.360668,inf,4.577636,2.000000,0.000000
3,4,49.595890,21.0,45,28,0.742195,0.750000,9.155272,3.023716,2.799417
4,5,70.729141,35.0,72,41,0.731584,0.853659,10.496030,3.885753,2.795443
...,...,...,...,...,...,...,...,...,...,...
177,178,150.361160,113.0,252,131,0.751740,0.862595,17.755183,5.887841,5.312459
178,179,13.220732,4.0,6,0,0.921726,inf,4.577636,2.000000,0.000000
179,180,338.516724,262.0,1050,447,0.584931,0.586130,42.634767,14.332246,6.821052
180,181,80.219513,74.0,126,87,1.062573,0.850575,14.843629,5.963269,3.221366


In [9]:
features_porespy.to_csv("../../measurements/cpfl/01-b-cpfl-porespy.csv", index = False)